In [ ]:
pip install beautifulsoup4


In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import glob

In [ ]:
datos = []

In [ ]:
# busca todos los archivos .html guardados en el directorio donde está esta script
for file in glob.glob("*.html"):
    with open(file, 'r', encoding="utf8") as html_file:
        content = html_file.read()
        soup = BeautifulSoup(content, 'lxml')

    bloques = soup.find_all('article', attrs={"id": "modern-variant-card"})

    for bloque in bloques:
        product = ""
        seller = ""
        brand = ""
        financial_mode = ""
        financing_amount = 0
        price = 0
        sale_price = 0

        valor = bloque.find('h6',  class_="d-inline-block")
        if valor:
            product = valor.text

        valor = bloque.find('div',  class_="shop-panel d-block mt-auto")
        if valor:
            seller = valor.text.replace("-", "").strip()

        valor = bloque.find('small',  class_="ng-star-inserted")
        if valor:
            f = valor.text.split("$")
            financial_mode = f[0]
            financing_amount = float(f[1].replace(
                "$", "").replace(".", "").replace(",", "."))

        valor = bloque.find(
            'div',  class_="manufacturer-panel d-block mt-auto")
        if valor:
            brand = valor.text.strip()

        # Trata de recuperar los precios unificados, si no encuenta valor, es porque vienen por separado, precio de oferta y precio regular
        valor = bloque.find(
            'span',  class_="price sale-price ng-star-inserted")
        if valor:
            price = float(valor.text.replace(
                "$", "").replace(".", "").replace(",", "."))
            sale_price = float(valor.text.replace(
                "$", "").replace(".", "").replace(",", "."))
        else:
            valor = bloque.find('span',  class_="price ng-star-inserted")
            if valor:
                price = float(valor.text.replace(
                    "$", "").replace(".", "").replace(",", "."))

            valor = bloque.find('span',  class_="sale-price ng-star-inserted")
            if valor:
                sale_price = float(valor.text.replace(
                    "$", "").replace(".", "").replace(",", "."))

        datos.append({"product": product, "brand": brand, "seller": seller, "financial_mode": financial_mode,"financing_amount": financing_amount, "price": price, "sale_price": sale_price})


print(datos)

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(datos)
df.index = df.index + 1
df.to_csv(r'bna.csv', index=True, header=True)